## Step 1 : Web Scraping 

In [23]:
#Import dependencies 
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pandas as pd
from random import randint
from time import sleep 

In [24]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\eenab\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache




In [25]:
#Create empty dictionary to hold all variables 
mars_data = {}

# title : string
# news_p : string
# image :string
# facts_table : string
# hemispheres: list of dictionaries with title and img_url


## NASA Mars News

In [26]:
# Visit Nasa news url through splinter module
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
sleep(randint(1,10))

In [27]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(browser.html, 'html.parser')

In [28]:
#Find latest news title 
news_title = soup.find_all('div', class_='content_title')
news_title[1].get_text()
title = news_title[1].get_text()
mars_data["title"] = title
title

'Testing Proves Its Worth With Successful Mars Parachute Deployment'

In [29]:
#Find latest news description
news_article = soup.find('div', class_='article_teaser_body')
news_p= news_article.get_text()
mars_data["news_p"] = news_p
news_p

'The giant canopy that helped land Perseverance on Mars was tested here on Earth at NASA’s Wallops Flight Facility in Virginia.'

In [30]:
# browser.quit()

## Featured Image

In [31]:
# Visit JPL Mars Space Images
jpl_img_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_img_url)
sleep(randint(1,10))


In [32]:
# Sort for latest Mars photos in gallery 
browser.find_by_id('filter_Mars').first.click()
sleep(randint(1,10))

In [33]:
#Set up new html/bs
featured_img_html = browser.html
img_soup = BeautifulSoup(featured_img_html, 'html')
featured_img_url = img_soup.find('img', class_="BaseImage object-contain")["src"]
print(featured_img_url)
mars_data["image"] = featured_img_url

https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24415.2e16d0ba.fill-400x400-c50.jpg


## Mars Facts

In [34]:
# Visit Mars facts url through splinter module
facts_url = 'https://space-facts.com/mars/'
browser.visit(facts_url)
sleep(randint(1,10))

In [35]:
# Scrape the table containing facts about the planet including Diameter, Mass, etc.and turn into HTML table string
facts_table = pd.read_html(facts_url)

mars_facts= facts_table[0]
# Rename columns 
mars_facts.columns = ['Description', 'Value']

mars_facts_html = mars_facts.to_html(index = False)

#insert into dictionary
mars_data["facts_table"]= mars_facts_html 


## Mars Hemispheres

In [36]:
#Set an empty list to hold urls
hemis_img_urls = []
    # {"title": "Valles Marineris Hemisphere", "img_url": "..."},
    # {"title": "Cerberus Hemisphere", "img_url": "..."},
    # {"title": "Schiaparelli Hemisphere", "img_url": "..."},
    # {"title": "Syrtis Major Hemisphere", "img_url": "..."},


In [37]:
#Visit main site
hemis_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemis_url)
sleep(randint(1,10))

In [38]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [39]:
#Locate each of the divs with hemisphere info
base_url = "https://astrogeology.usgs.gov"

results = soup.find_all("div", class_='description')
results

[<div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div>,
 <div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><h3>Schiaparelli Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 35 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Schiaparelli hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. The images were acquired in 1980 during early northern…</p></div>,
 <div class="description"><a 

In [40]:
#Below finds one link, used as an example
results[0].find('a')['href']

'/search/map/Mars/Viking/cerberus_enhanced'

In [41]:
#Runs through each hemisphere div, clicks in to find link to full size image URL, title of Hemisphere, and appends the dictionary
for result in results:
    
    hemi_title = result.find('h3').text

    img_page_url = base_url + result.find('a')['href']
    browser.visit(img_page_url)
    
    html = browser.html
    img_page_soup = BeautifulSoup(html, 'html.parser')
    
    hemi_img_url = img_page_soup.find('ul').li.a['href']
    
    hemi_img_dict = {'title': hemi_title, 
                'img_url': hemi_img_url}
    
    hemis_img_urls.append(hemi_img_dict)


In [42]:
hemis_img_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [43]:
#Insert into dictionary
mars_data["hemispheres"]= hemis_img_urls

In [48]:
mars_data

{'title': 'Testing Proves Its Worth With Successful Mars Parachute Deployment',
 'news_p': 'The giant canopy that helped land Perseverance on Mars was tested here on Earth at NASA’s Wallops Flight Facility in Virginia.',
 'image': 'https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24415.2e16d0ba.fill-400x400-c50.jpg',
 'facts_table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Description</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9

In [45]:
browser.quit()